### Stage 1:
Perform statistical parsing/tagging on a document in JSON format

INPUTS: JSON doc for the text input  
OUTPUT: JSON format `ParsedGraf(id, sha1, graf)`

In [1]:
import ast
from cmd_pytextrank import pytextrank
import sys

In [2]:
def get_key_terms(text_doc):
    path_stage1 = "o1.json"
    with open(path_stage1, 'w') as f:
        for graf in pytextrank.parse_doc(pytextrank.json_iter(text_doc)):
            f.write("%s\n" % pytextrank.pretty_print(graf._asdict()))

    graph, ranks = pytextrank.text_rank(path_stage1)
    pytextrank.render_ranks(graph, ranks)

    result = []
    count = 0;
    for rl in pytextrank.normalize_key_phrases(path_stage1, ranks):
        count += 1
#         if count>5: break
#         print(pytextrank.pretty_print(rl))
        result.append(ast.literal_eval(pytextrank.pretty_print(rl))[:2])
        
    return result

In [25]:
text_doc = "./test_data/news_group1.json"
key_terms = get_key_terms(text_doc)
print(len(key_terms))

42


In [7]:
import spacy
nlp = spacy.load('en_core_web_md')

In [26]:
# words = []
# for x in key_terms:
#     for y in x[0].split(" "):
#         words.append(y)
# print(len(words))

50


In [21]:
# nlp('apple').vector

In [16]:
from nltk.cluster import KMeansClusterer
import nltk
NUM_CLUSTERS=5
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(word_vectors, assign_clusters=True)
print (assigned_clusters)

TypeError: cluster() got an unexpected keyword argument 'avoid_empty_cluster'

In [17]:
from sklearn import cluster
from sklearn import metrics
kmeans = cluster.KMeans(n_clusters=NUM_CLUSTERS)
kmeans.fit(word_vectors)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

In [20]:
print ("Cluster id labels for inputted data")
print (labels)
print ("Centroids data")
print (centroids)
 
print ("Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):")
# print (kmeans.score(X))

Cluster id labels for inputted data
[2 1 2 4 2 0 0 0 0 3 1 0 4 0 0 1 4 4 0 0 4 0 0 0 2 2 2 4 4 0 0 2 3 4 4 2 0
 2 0 0 3 4]
Centroids data
[[ 0.07589516  0.11340641  0.04369762 ...  0.08499742 -0.07448617
   0.00815011]
 [ 0.1818      0.1048935   0.0406     ...  0.10874875  0.16686051
   0.32704001]
 [ 0.32438943  0.33282678 -0.07430638 ... -0.05602555  0.20864282
   0.19725547]
 [ 0.2693588  -0.03567133  0.15932334 ...  0.17990966  0.0201788
  -0.15116334]
 [-0.04187154  0.06686923 -0.39417037 ... -0.04828533  0.01197026
   0.37658137]]
Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):


In [28]:
for i, word in enumerate(key_terms):
    print(word, labels[i])

['car', 0.08623640497600257] 2
['engine specs', 0.05427824354964387] 1
['looking car', 0.05208175460255698] 2
['model name', 0.047578672329223304] 4
['sports car', 0.043118202488001287] 2
['production', 0.04090044141323622] 0
['early 70s', 0.04031687230572975] 0
['years', 0.04009683900070784] 0
['other day', 0.03508202358741993] 0
['college park', 0.03471230409210795] 3
['specs', 0.027139121774821934] 1
['lines', 0.026747576710051287] 0
['looking', 0.02604087730127849] 4
['history', 0.02554701357070712] 0
['body', 0.024240950693103475] 0
['engine', 0.024023955480460015] 1
['name', 0.023789336164611652] 4
['tellme', 0.023238034492637467] 4
['rest', 0.020497370301711896] 0
['70s', 0.020158436152864875] 0
['anyone', 0.019317106329831187] 4
['bricklin', 0.019317106329831187] 0
['made', 0.01777142710639794] 0
['day', 0.017541011793709965] 0
['front bumper', 0.017500737001061792] 2
['park', 0.017356152046053976] 2
['model', 0.016694268845221214] 2
['subject', 0.015693877984740062] 4
['e-mail

In [ ]:
import spacy

# Load the spacy vocabulary
nlp = spacy.load("en_core_web_md")

In [54]:
news_topics = []
with open('word_list.csv', 'r') as f:
    for x in f.read().split(','):
        news_topics.append(x)
print(news_topics)

['apple', 'mango', 'banana\n']


In [58]:
from scipy.spatial import distance
import numpy as np


# Format the input vector for use in the distance function
# In this case we will artificially create a word vector from a real word ("frog")
# but any derived word vector could be used
p = np.array([centroids[4]])
print(len(p))
# Format the vocabulary for use in the distance function
ids = [x for x in nlp.vocab.vectors.keys()]
print(len(ids))

vectors = [nlp.vocab.vectors[x] for x in ids]
vectors = np.array(vectors)

# *** Find the closest word below ***
closest_index = distance.cdist(p, vectors).argmin()
print(closest_index)
word_id = ids[closest_index]
output_word = nlp.vocab[word_id].text
print(output_word)
# output_word is identical, or very close, to the input word

1
684830
156
sure


In [63]:
max(ids)

18446743695545242228

In [43]:
from scipy.spatial import distance
import numpy as np


# Format the input vector for use in the distance function
# In this case we will artificially create a word vector from a real word ("frog")
# but any derived word vector could be used
p = np.array([centroids[4]])
print(len(p))
# Format the vocabulary for use in the distance function
ids = [x for x in nlp.vocab.vectors.keys()]
print(len(ids))

vectors = [nlp.vocab.vectors[x] for x in ids]
vectors = np.array(vectors)

# *** Find the closest word below ***
closest_index = distance.cdist(p, vectors).argmin()
word_id = ids[closest_index]
output_word = nlp.vocab[word_id].text
print(output_word)
# output_word is identical, or very close, to the input word

1
684830
sure


In [36]:
output_word

'same'

In [71]:
count = 0
for x in nlp.vocab:
    count += 1
    if count >= 100: break
    print(x)